In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from sklearn.feature_extraction.text import CountVectorizer, \
HashingVectorizer, TfidfVectorizer
from sklearn.datasets import fetch_20newsgroups
from sklearn.decomposition import TruncatedSVD
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import make_union, make_pipeline
from sklearn.svm import SVC, SVR
from sklearn import svm

import string

#  Let set option to 300 each 

In [2]:
pd.set_option('display.height', 500)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

height has been deprecated.



# Load the Data

In [3]:
data = pd.read_csv('reddit_posts.csv')

In [4]:
data.head()

,adserver_click_url,adserver_imp_pixel,archived,author,author_flair_css_class,author_flair_text,contest_mode,created_utc,disable_comments,distinguished,domain,downs,edited,gilded,hide_score,href_url,id,imp_pixel,is_self,link_flair_css_class,link_flair_text,locked,media,media_embed,mobile_ad_url,name,num_comments,original_link,over_18,permalink,post_hint,preview,promoted,promoted_by,promoted_display_name,promoted_url,quarantine,retrieved_on,saved,score,secure_media,secure_media_embed,selftext,spoiler,stickied,subreddit,subreddit_id,third_party_tracking,third_party_tracking_2,thumbnail,title,ups,url
0,NaN,NaN,False,johnnyawesome0,NaN,NaN,False,1480697304,NaN,NaN,self.techsupport,0.0,False,0.0,False,NaN,5g49s2,NaN,True,NaN,NaN,False,NaN,{},NaN,t3_5g49s2,1.0,NaN,False,/r/techsupport/comments/5g49s2/help_with_audio...,NaN,NaN,NaN,NaN,NaN,NaN,False,1.484297e+09,False,1.0,NaN,{},I have a Sony surround sound system for a blu-...,False,False,techsupport,t5_2qioo,NaN,NaN,self,Help with audio set-up,1.0,https://www.reddit.com/r/techsupport/comments/...
1,NaN,NaN,False,Silverfin113,NaN,NaN,False,1480697424,NaN,NaN,self.learnprogramming,0.0,False,0.0,False,NaN,5g4a5p,NaN,True,NaN,NaN,False,NaN,{},NaN,t3_5g4a5p,8.0,NaN,False,/r/learnprogramming/comments/5g4a5p/optimizing...,NaN,NaN,NaN,NaN,NaN,NaN,False,1.484297e+09,False,23.0,NaN,{},I've written what seems to be a prohibitively ...,False,False,learnprogramming,t5_2r7yd,NaN,NaN,self,Optimizing code for speed,23.0,https://www.reddit.com/r/learnprogramming/comm...
2,NaN,NaN,False,bookbooksbooks,NaN,NaN,False,1480697613,NaN,NaN,self.gamedev,0.0,False,0.0,False,NaN,5g4att,NaN,True,discussion cat-talk,Discussion,False,NaN,{},NaN,t3_5g4att,5.0,NaN,False,/r/gamedev/comments/5g4att/seeking_tales_of_de...,NaN,NaN,NaN,NaN,NaN,NaN,False,1.484297e+09,False,12.0,NaN,{},"I'm writing an article called ""Video Games Tha...",False,False,gamedev,t5_2qi0a,NaN,NaN,self,Seeking Tales of Development Woe (and Triumph)...,12.0,https://www.reddit.com/r/gamedev/comments/5g4a...
3,NaN,NaN,False,[deleted],NaN,NaN,False,1480697634,NaN,NaN,self.learnprogramming,0.0,1480698462,0.0,False,NaN,5g4awr,NaN,True,solved,Solved,False,NaN,{},NaN,t3_5g4awr,9.0,NaN,False,/r/learnprogramming/comments/5g4awr/java_findi...,NaN,NaN,NaN,NaN,NaN,NaN,False,1.484297e+09,False,0.0,NaN,{},[deleted],False,False,learnprogramming,t5_2r7yd,NaN,NaN,default,[Java] Finding smallest value in an array,0.0,https://www.reddit.com/r/learnprogramming/comm...
4,NaN,NaN,False,caffeine_potent,NaN,NaN,False,1480697748,NaN,NaN,self.learnpython,0.0,1480709138,0.0,False,NaN,5g4bcr,NaN,True,NaN,NaN,False,NaN,{},NaN,t3_5g4bcr,12.0,NaN,False,/r/learnpython/comments/5g4bcr/currying_functi...,NaN,NaN,NaN,NaN,NaN,NaN,False,1.484297e+09,False,6.0,NaN,{},I have the following representation of argumen...,False,False,learnpython,t5_2r8ot,NaN,NaN,self,currying functions using functools,6.0,https://www.reddit.com/r/learnpython/comments/...


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26688 entries, 0 to 26687
Data columns (total 53 columns):
adserver_click_url        0 non-null float64
adserver_imp_pixel        0 non-null float64
archived                  26688 non-null bool
author                    26688 non-null object
author_flair_css_class    435 non-null object
author_flair_text         351 non-null object
contest_mode              26688 non-null bool
created_utc               26688 non-null int64
disable_comments          0 non-null float64
distinguished             85 non-null object
domain                    26688 non-null object
downs                     26688 non-null float64
edited                    26688 non-null object
gilded                    26688 non-null float64
hide_score                26688 non-null bool
href_url                  0 non-null float64
id                        26688 non-null object
imp_pixel                 0 non-null float64
is_self                   26688 non-null bool
link_fla

In [6]:
data.columns

Index(['adserver_click_url', 'adserver_imp_pixel', 'archived', 'author', 'author_flair_css_class', 'author_flair_text', 'contest_mode', 'created_utc', 'disable_comments', 'distinguished', 'domain', 'downs', 'edited', 'gilded', 'hide_score', 'href_url', 'id', 'imp_pixel', 'is_self', 'link_flair_css_class', 'link_flair_text', 'locked', 'media', 'media_embed', 'mobile_ad_url', 'name', 'num_comments', 'original_link', 'over_18', 'permalink', 'post_hint', 'preview', 'promoted', 'promoted_by', 'promoted_display_name', 'promoted_url', 'quarantine', 'retrieved_on', 'saved', 'score', 'secure_media', 'secure_media_embed', 'selftext', 'spoiler', 'stickied', 'subreddit', 'subreddit_id', 'third_party_tracking', 'third_party_tracking_2', 'thumbnail', 'title', 'ups', 'url'], dtype='object')

In [7]:
# data.subreddit.value_counts()[0:10].plot(kind='bar')
# plt.title('Top 10 SubReddit Posts', fontsize=25)

In [8]:
#Find the missing values in the dataset
all_null_columns = data.columns[data.isnull().sum()==len(data)]

In [9]:
data.isnull().sum().sum()

569801

In [10]:
#Find the missing values in the dataset
all_null_columns = data.columns[data.isnull().sum()==len(data)]
all_null_columns

Index(['adserver_click_url', 'adserver_imp_pixel', 'disable_comments', 'href_url', 'imp_pixel', 'mobile_ad_url', 'original_link', 'promoted', 'promoted_by', 'promoted_display_name', 'promoted_url', 'third_party_tracking', 'third_party_tracking_2'], dtype='object')

In [11]:
#Dropping the Null Columns
data.drop(all_null_columns,axis=1, inplace=True)

In [12]:
data.head()

,archived,author,author_flair_css_class,author_flair_text,contest_mode,created_utc,distinguished,domain,downs,edited,gilded,hide_score,id,is_self,link_flair_css_class,link_flair_text,locked,media,media_embed,name,num_comments,over_18,permalink,post_hint,preview,quarantine,retrieved_on,saved,score,secure_media,secure_media_embed,selftext,spoiler,stickied,subreddit,subreddit_id,thumbnail,title,ups,url
0,False,johnnyawesome0,NaN,NaN,False,1480697304,NaN,self.techsupport,0.0,False,0.0,False,5g49s2,True,NaN,NaN,False,NaN,{},t3_5g49s2,1.0,False,/r/techsupport/comments/5g49s2/help_with_audio...,NaN,NaN,False,1.484297e+09,False,1.0,NaN,{},I have a Sony surround sound system for a blu-...,False,False,techsupport,t5_2qioo,self,Help with audio set-up,1.0,https://www.reddit.com/r/techsupport/comments/...
1,False,Silverfin113,NaN,NaN,False,1480697424,NaN,self.learnprogramming,0.0,False,0.0,False,5g4a5p,True,NaN,NaN,False,NaN,{},t3_5g4a5p,8.0,False,/r/learnprogramming/comments/5g4a5p/optimizing...,NaN,NaN,False,1.484297e+09,False,23.0,NaN,{},I've written what seems to be a prohibitively ...,False,False,learnprogramming,t5_2r7yd,self,Optimizing code for speed,23.0,https://www.reddit.com/r/learnprogramming/comm...
2,False,bookbooksbooks,NaN,NaN,False,1480697613,NaN,self.gamedev,0.0,False,0.0,False,5g4att,True,discussion cat-talk,Discussion,False,NaN,{},t3_5g4att,5.0,False,/r/gamedev/comments/5g4att/seeking_tales_of_de...,NaN,NaN,False,1.484297e+09,False,12.0,NaN,{},"I'm writing an article called ""Video Games Tha...",False,False,gamedev,t5_2qi0a,self,Seeking Tales of Development Woe (and Triumph)...,12.0,https://www.reddit.com/r/gamedev/comments/5g4a...
3,False,[deleted],NaN,NaN,False,1480697634,NaN,self.learnprogramming,0.0,1480698462,0.0,False,5g4awr,True,solved,Solved,False,NaN,{},t3_5g4awr,9.0,False,/r/learnprogramming/comments/5g4awr/java_findi...,NaN,NaN,False,1.484297e+09,False,0.0,NaN,{},[deleted],False,False,learnprogramming,t5_2r7yd,default,[Java] Finding smallest value in an array,0.0,https://www.reddit.com/r/learnprogramming/comm...
4,False,caffeine_potent,NaN,NaN,False,1480697748,NaN,self.learnpython,0.0,1480709138,0.0,False,5g4bcr,True,NaN,NaN,False,NaN,{},t3_5g4bcr,12.0,False,/r/learnpython/comments/5g4bcr/currying_functi...,NaN,NaN,False,1.484297e+09,False,6.0,NaN,{},I have the following representation of argumen...,False,False,learnpython,t5_2r8ot,self,currying functions using functools,6.0,https://www.reddit.com/r/learnpython/comments/...


# Deleted SelfText 

In [13]:
#find the SelfText columns which have the word deleted in them

data.loc[data['selftext']=='[deleted]',:][:5]

,archived,author,author_flair_css_class,author_flair_text,contest_mode,created_utc,distinguished,domain,downs,edited,gilded,hide_score,id,is_self,link_flair_css_class,link_flair_text,locked,media,media_embed,name,num_comments,over_18,permalink,post_hint,preview,quarantine,retrieved_on,saved,score,secure_media,secure_media_embed,selftext,spoiler,stickied,subreddit,subreddit_id,thumbnail,title,ups,url
3,False,[deleted],NaN,NaN,False,1480697634,NaN,self.learnprogramming,0.0,1480698462,0.0,False,5g4awr,True,solved,Solved,False,NaN,{},t3_5g4awr,9.0,False,/r/learnprogramming/comments/5g4awr/java_findi...,NaN,NaN,False,1.484297e+09,False,0.0,NaN,{},[deleted],False,False,learnprogramming,t5_2r7yd,default,[Java] Finding smallest value in an array,0.0,https://www.reddit.com/r/learnprogramming/comm...
6,False,[deleted],NaN,NaN,False,1480697781,NaN,self.Python,0.0,False,0.0,False,5g4bg2,True,NaN,NaN,False,NaN,{},t3_5g4bg2,5.0,False,/r/Python/comments/5g4bg2/how_can_i_use_differ...,NaN,NaN,False,1.484297e+09,False,0.0,NaN,{},[deleted],False,False,Python,t5_2qh0y,default,How can I use (different 'groups' of 8 differe...,0.0,https://www.reddit.com/r/Python/comments/5g4bg...
10,False,[deleted],NaN,NaN,False,1480698138,NaN,amazon.com,0.0,False,0.0,False,5g4clp,False,NaN,NaN,False,NaN,{},t3_5g4clp,0.0,False,/r/technology/comments/5g4clp/do_you_life_adve...,NaN,NaN,False,1.484297e+09,False,1.0,NaN,{},[deleted],False,False,technology,t5_2qh16,default,Do you life adventure? Don't forget to take th...,1.0,https://www.amazon.com/Paracord-Survival-Brace...
11,False,[deleted],NaN,NaN,False,1480698278,NaN,popsci.com,0.0,False,0.0,False,5g4d5c,False,general,AI,False,NaN,{},t3_5g4d5c,0.0,False,/r/technology/comments/5g4d5c/reuters_built_a_...,NaN,NaN,False,1.484297e+09,False,3.0,NaN,{},[deleted],False,False,technology,t5_2qh16,default,Reuters built a bot that can identify real new...,3.0,http://www.popsci.com/artificial-intelligence-...
17,False,[deleted],NaN,NaN,False,1480698834,NaN,self.techsupport,0.0,False,0.0,False,5g4f2j,True,NaN,NaN,False,NaN,{},t3_5g4f2j,12.0,False,/r/techsupport/comments/5g4f2j/connect_pc_to_l...,NaN,NaN,False,1.484297e+09,False,1.0,NaN,{},[deleted],False,False,techsupport,t5_2qioo,default,Connect PC to laptop,1.0,https://www.reddit.com/r/techsupport/comments/...


In [14]:
print (len(data.loc[data['selftext']=='[deleted]',:]))

5269


In [15]:
data['selftext'][0]

'I have a Sony surround sound system for a blu-ray player with the 4.2mm connectors. Is there a way I can play all my other devices through my speakers. What kind of connectors and amps do I need to get it to work'

In [16]:
#Subreddit Category with the most UPS Votes
#specific reddit group that has the most UPS votes

data.groupby(by='subreddit',axis=0).ups.sum().sort_values(ascending=False)

subreddit
technology          79953.0
learnprogramming    38286.0
gamedev             18855.0
techsupport         18515.0
linux               11989.0
learnpython          9852.0
Python               6006.0
web_design           4631.0
engineering          3939.0
javascript           2136.0
PHP                  1549.0
java                 1538.0
compsci              1390.0
javahelp             1159.0
iOSProgramming       1042.0
dailyprogrammer       753.0
swift                 735.0
computerscience       708.0
csshelp               629.0
cpp                   607.0
django                579.0
ruby                  520.0
programming           511.0
netsec                498.0
css                   309.0
flask                 230.0
html5                 171.0
perl                  168.0
HTML                  146.0
jquery                118.0
pygame                114.0
lisp                   90.0
coding                 65.0
programmer             28.0
IPython                27.0
inventwith

In [17]:
# data.groupby(by='subreddit',axis=0).ups.sum().sort_values(ascending=False)[0:9].plot(kind='bar')
# plt.title('Top 10 SubReddits with the most Up Votes', fontsize=25)

In [18]:
clean_data = data.loc[data['selftext']!='[deleted]',:]
clean_data.head()

,archived,author,author_flair_css_class,author_flair_text,contest_mode,created_utc,distinguished,domain,downs,edited,gilded,hide_score,id,is_self,link_flair_css_class,link_flair_text,locked,media,media_embed,name,num_comments,over_18,permalink,post_hint,preview,quarantine,retrieved_on,saved,score,secure_media,secure_media_embed,selftext,spoiler,stickied,subreddit,subreddit_id,thumbnail,title,ups,url
0,False,johnnyawesome0,NaN,NaN,False,1480697304,NaN,self.techsupport,0.0,False,0.0,False,5g49s2,True,NaN,NaN,False,NaN,{},t3_5g49s2,1.0,False,/r/techsupport/comments/5g49s2/help_with_audio...,NaN,NaN,False,1.484297e+09,False,1.0,NaN,{},I have a Sony surround sound system for a blu-...,False,False,techsupport,t5_2qioo,self,Help with audio set-up,1.0,https://www.reddit.com/r/techsupport/comments/...
1,False,Silverfin113,NaN,NaN,False,1480697424,NaN,self.learnprogramming,0.0,False,0.0,False,5g4a5p,True,NaN,NaN,False,NaN,{},t3_5g4a5p,8.0,False,/r/learnprogramming/comments/5g4a5p/optimizing...,NaN,NaN,False,1.484297e+09,False,23.0,NaN,{},I've written what seems to be a prohibitively ...,False,False,learnprogramming,t5_2r7yd,self,Optimizing code for speed,23.0,https://www.reddit.com/r/learnprogramming/comm...
2,False,bookbooksbooks,NaN,NaN,False,1480697613,NaN,self.gamedev,0.0,False,0.0,False,5g4att,True,discussion cat-talk,Discussion,False,NaN,{},t3_5g4att,5.0,False,/r/gamedev/comments/5g4att/seeking_tales_of_de...,NaN,NaN,False,1.484297e+09,False,12.0,NaN,{},"I'm writing an article called ""Video Games Tha...",False,False,gamedev,t5_2qi0a,self,Seeking Tales of Development Woe (and Triumph)...,12.0,https://www.reddit.com/r/gamedev/comments/5g4a...
4,False,caffeine_potent,NaN,NaN,False,1480697748,NaN,self.learnpython,0.0,1480709138,0.0,False,5g4bcr,True,NaN,NaN,False,NaN,{},t3_5g4bcr,12.0,False,/r/learnpython/comments/5g4bcr/currying_functi...,NaN,NaN,False,1.484297e+09,False,6.0,NaN,{},I have the following representation of argumen...,False,False,learnpython,t5_2r8ot,self,currying functions using functools,6.0,https://www.reddit.com/r/learnpython/comments/...
5,False,uzbek1993,NaN,NaN,False,1480697775,NaN,self.learnprogramming,0.0,False,0.0,False,5g4bfi,True,NaN,NaN,False,NaN,{},t3_5g4bfi,1.0,False,/r/learnprogramming/comments/5g4bfi/text_edito...,NaN,NaN,False,1.484297e+09,False,1.0,NaN,{},I am about to create a website where users use...,False,False,learnprogramming,t5_2r7yd,self,Text Editor integration,1.0,https://www.reddit.com/r/learnprogramming/comm...


In [19]:
#combine the SelfText, Title and Subreddit columns
self_text = clean_data['selftext']
title = clean_data['title']
subreddit = clean_data['subreddit']

In [20]:
clean_data['aggregated_column'] = clean_data['selftext'] + ' ' + clean_data['subreddit'] + ' ' +  clean_data['title'] 

C:\Users\Pernesso\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [21]:
# Linear Regression Model with combine_column to make the prediction for ups (upvotes)

# Vectorizer

In [22]:
X = clean_data['aggregated_column']
y = clean_data['ups']

vt = CountVectorizer(stop_words='english')
X_vt = vt.fit(X)
X_vt_transform = X_vt.transform(X)

In [24]:
#Creating a dataframe using the transformed Values
x_vt_df = pd.DataFrame(X_vt_transform.todense(), columns=X_vt.get_feature_names(), index=clean_data.index)
x_vt_df.head()

MemoryError: 

In [ ]:
# len(x_vt_df.columns)

In [ ]:
#Too many unwanted columns, let's truncate'

In [ ]:
truncatedSVD = TruncatedSVD(n_components=2000)
truncatedSVD.fit(X_vt_transform)

In [ ]:
plt.plot(range(2000), truncatedSVD.explained_variance_ratio_.cumsum())

In [ ]:
truncatedSVD_2k = TruncatedSVD(n_components=1000)
truncatedSVD_2k.fit(x_vt_df)

In [ ]:
truncatedSVD_2k_transform = truncatedSVD_2k.transform(x_vt_df)

In [ ]:
#Using the Regressor to Predict the Up Votes
y = clean_data['score']
X = truncatedSVD_2k_transform

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2)

# Linear Regresssion

In [ ]:
lr = LinearRegression()
lr.fit(X_train, y_train)
lr.score(X_train,y_train)

In [ ]:

#lr.fit(X_train, y_train)

In [ ]:

#lr.score(X_train,y_train)

In [ ]:
lr.score(X_test,y_test)

This is a bad model! We score bad on train and worse of test

In [ ]:
## Model 2: Random Forest Regression Model with aggregated column to predict the ups (up votes)
The model uses a Vectorizer, SVD and a simple Linear Regression to build a base model

In [ ]:

rfr = RandomForestRegressor(n_estimators=100, max_depth=15, n_jobs=-1)
rfr.fit(X_train, y_train)
rfr.score(X_train, y_train)
rfr.score(X_test, y_test)

In [ ]:
#rfr.fit(X_train, y_train)

In [ ]:
#rfr.score(X_train, y_train)

In [ ]:
#rfr.score(X_test, y_test)

In [ ]:
# Because regressor models for the aggregated columns have scored poorly
# let's use the selftext column and apply CountVectorizer to predict the scores.
X = clean_data['selftext']
vt = CountVectorizer(stop_words='english')
X_vt = vt.fit(X)
X_vt_transform = X_vt.transform(X)

In [ ]:
'''Creating a dataframe using the transformed Values'''
x_vt_df = pd.DataFrame(X_vt_transform.todense(), columns=X_vt.get_feature_names(), index=clean_data.index)
x_vt_df.head()

In [ ]:
'''Using 1500 components to find the new fit. In the earlier cells we had used a value of 1000'''

truncatedSVD_2k = TruncatedSVD(n_components=1500)
truncatedSVD_2k.fit(x_vt_df)
truncatedSVD_2k_transform = truncatedSVD_2k.transform(x_vt_df)

In [ ]:
#truncatedSVD_2k_transform = truncatedSVD_2k.transform(x_vt_df)

In [ ]:
# Predictors and the Target
y = clean_data['score']
X = truncatedSVD_2k_transform

In [ ]:

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2)

In [ ]:

lr = LinearRegression()
lr.fit(X_train, y_train)
lr.score(X_train,y_train)
lr.score(X_test,y_test)

In [ ]:

#lr.fit(X_train, y_train)

In [ ]:

#lr.score(X_train,y_train)

In [ ]:

#lr.score(X_test,y_test)

In [ ]:
#let's build out a classifier model that would predict whether a post belongs to Tech Support or not based on a #simple model

In [ ]:
#Model 3: Logisitic Regression model to predict the SubReddit Category as Techsupport v/s Non-Tech Support

In [ ]:
#Flagging any article that has TechSupport in its category as 1 and 0 otherwise

clean_data['Is_Techsupport'] = clean_data['subreddit'].apply(lambda x: 1 if x == 'techsupport' else 0)

In [ ]:
clean_data.head()

# Vectorizer

In [ ]:
X = clean_data['selftext']
vt = CountVectorizer(stop_words='english')
X_vt = vt.fit(X)
X_vt_transform = X_vt.transform(X)

In [ ]:
'''Creating a dataframe using the transformed Values'''
x_vt_df = pd.DataFrame(X_vt_transform.todense(), columns=X_vt.get_feature_names(), index=clean_data.index)
x_vt_df.head()

In [ ]:
# Applying SVD
truncatedSVD_200 = TruncatedSVD(n_components=200)
truncatedSVD_200.fit(x_vt_df)
truncatedSVD_200_transform = truncatedSVD_200.transform(x_vt_df)

In [ ]:
#truncatedSVD_200_transform = truncatedSVD_200.transform(x_vt_df)

In [ ]:
#Set the Predictors and the Target Variable

y = clean_data['Is_Techsupport']
X = truncatedSVD_200_transform

In [ ]:

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2)

# Logistic Regression

In [ ]:
# using Grid Search for HyperParameter tuning 

from sklearn.linear_model import LogisticRegression

lr_params = {
    'penalty':['l1','l2'],
    'C':np.logspace(-4, 2, 20),
    'solver':['liblinear']
}

logreg_gs = GridSearchCV(LogisticRegression(), lr_params, cv=5, verbose=1, n_jobs=-1)
logreg_gs.fit(X_train, y_train)

In [ ]:

logreg_gs.best_estimator_
logreg_gs.best_params_
logreg_gs.best_score_

In [ ]:

#logreg_gs.best_params_

In [ ]:

#logreg_gs.best_score_

In [ ]:

logreg_optimal = LogisticRegression(C=0.5, penalty='l2', solver='liblinear',max_iter=5000, n_jobs = -1)

In [ ]:

logreg_optimal.fit(X_train, y_train)
print ('Training Score: ', logreg_optimal.score(X_train, y_train))
print ('Testing Score: ', logreg_optimal.score(X_test, y_test))

In [ ]:
#print ('Training Score: ', logreg_optimal.score(X_train, y_train))

In [ ]:
#print ('Testing Score: ', logreg_optimal.score(X_test, y_test))

In [ ]:
confusion_matrix(y_train, logreg_optimal.predict(X_train))
confusion_matrix(y_test, logreg_optimal.predict(X_test))
print (classification_report(y_train, logreg_optimal.predict(X_train)))
print (classification_report(y_test, logreg_optimal.predict(X_test)))

In [ ]:

#confusion_matrix(y_test, logreg_optimal.predict(X_test))

In [ ]:
#print (classification_report(y_train, logreg_optimal.predict(X_train)))

In [ ]:
#print (classification_report(y_test, logreg_optimal.predict(X_test)))

In [ ]:
#This is a good classifier model with a score of ~0.9 on Train and Test sets. 
#Plotting the confusion matrix and the classification report also gives a good score of 0.95 on the training and 
#test sets. However we are going to try some more models like SVM to see 
#if we can further optimize the model for a better score 

# Model 4: Support Vector Machines (kernel = Linear)

### Linear SVM

In [ ]:
'''Instantiating an Linear SVM Model'''
from sklearn.svm import SVC
svm_linear = svm.SVC(kernel='linear')
svm_linear_fit = svm_linear.fit(X_train, y_train)
svm_linear_fit
svm_linear_fit.score(X_train, y_train)
svm_linear_fit.score(X_test, y_test)
cr = classification_report(y_train, svm_linear_fit.predict(X_train), digits=2)
print (cr)
cr = classification_report(y_test, svm_linear_fit.predict(X_test), digits=2)
print (cr

In [ ]:
'''Fitting the SVM model on the Training set'''
# svm_linear_fit = svm_linear.fit(X_train, y_train)
# svm_linear_fit

In [ ]:
'''Scoring the Model with Training set'''
# svm_linear_fit.score(X_train, y_train)

In [ ]:
'''Scoring the Model with Test set'''
# svm_linear_fit.score(X_test, y_test)

In [ ]:
'''Classification Report on the Training Set'''
# cr = classification_report(y_train, svm_linear_fit.predict(X_train), digits=2)
# print (cr)

In [ ]:
'''Clasification report on the Test Set'''
# cr = classification_report(y_test, svm_linear_fit.predict(X_test), digits=2)
# print (cr)

The linear SVM model scores around ~0.95 on the train and the test sets. We shall test the model with other variations of SVM model.

# Model 5: Support Vector Machines (kernel = RBF)

In [ ]:

from sklearn.svm import SVC
svm_rbf = svm.SVC(kernel='rbf', C=0.5)
svm_rbf_fit = svm_rbf.fit(X_train, y_train)
svm_rbf_fit
svm_rbf_fit.score(X_train, y_train)
svm_rbf_fit.score(X_test, y_test)
cr = classification_report(y_train, svm_rbf_fit.predict(X_train), digits=2)
cr = classification_report(y_test, svm_rbf_fit.predict(X_test), digits=2)
print (cr)

In [ ]:

# svm_rbf_fit = svm_rbf.fit(X_train, y_train)
# svm_rbf_fit

In [ ]:
'''Scoring the Model with Training set'''
# svm_rbf_fit.score(X_train, y_train)

In [ ]:
'''Scoring the Model with Test set'''
# svm_rbf_fit.score(X_test, y_test)

In [ ]:
'''Classification Report on the Training Set'''
# cr = classification_report(y_train, svm_rbf_fit.predict(X_train), digits=2)
# print (cr)

In [ ]:
'''Clasification report on the Test Set'''
# cr = classification_report(y_test, svm_rbf_fit.predict(X_test), digits=2)
# print (cr)

The overall score with the kernel model is close to the score we got on the Linear SVM model. Overall we see that the classifier models which tries to classify the article as a tech support article versus a non-technical support has a better performance than the regressor models 